In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
class A:

    def spam(self, x):
        print('A.spam:', x)

    def foo(self):
        print('A.foo')


class B:

    def __init__(self):
        self._a = A()

    def bar(self):
        print('B.bar')

    def __getattr__(self, name):
        return getattr(self._a, name)


"""

>>> b = B()
>>> b.bar()
B.bar
>>> b.spam(42)
A.spam: 42
"""

doctest.testmod()

TestResults(failed=0, attempted=3)

In [4]:
class Proxy:

    def __init__(self, obj):
        self._obj = obj

    def __getattr__(self, name):
        print('getattr:', name)
        return getattr(self._obj, name)

    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            print('setattr:', name, value)
            setattr(self._obj, name, value)

    def __delattr__(self, name):
        if name.startswith('_'):
            super().__delattr__(name)
        else:
            print('delattr:', name)
            delattr(self._obj, name)


class Spam:

    def __init__(self, x):
        self.x = x

    def bar(self, y):
        print('Spam.bar:', self.x, y)


"""

>>> s = Spam(2)
>>> p = Proxy(s)
>>> print(p.x)
getattr: x
2
>>> p.bar(3)
getattr: bar
Spam.bar: 2 3
>>> p.x = 37
setattr: x 37
>>> print(p.x)
getattr: x
37
>>> p._z = -1
>>> print(p._z)
-1
>>> s._z
Traceback (most recent call last):
    ...
AttributeError: 'Spam' object has no attribute '_z'
"""

doctest.testmod()

TestResults(failed=0, attempted=9)

In [5]:
from functools import partial


class A:

    def spam(self, x):
        print('A.spam:', x)

    def foo(self):
        print('A.foo')


class B:

    def __init__(self, *, delegate=None):
        self.delegate = delegate

    def __getattr__(self, name):
        if self.delegate is not None:
            return getattr(self.delegate, name)

    def spam(self, x):
        print('B.spam:', x)
        if self.delegate is not None:
            self.delegate.spam(x)

    def bar(self):
        print('B.bar')


"""

>>> a = A()
>>> b = B(delegate=a)
>>> b.spam(1)
B.spam: 1
A.spam: 1
>>> b.bar()
B.bar
>>> b2 = B()
>>> b2.spam(2)
B.spam: 2
"""

doctest.testmod()

TestResults(failed=0, attempted=6)

In [6]:
class ListLike:

    def __init__(self):
        self._items = []

    def __getattr__(self, name):
        return getattr(self._items, name)


"""

>>> a = ListLike()
>>> a.append(2)
>>> a.insert(0, 1)
>>> a.sort()
>>> len(a)
Traceback (most recent call last):
    ...
TypeError: object of type 'ListLike' has no len()
>>> a[0]
Traceback (most recent call last):
    ...
TypeError: 'ListLike' object is not subscriptable
"""

doctest.testmod()

TestResults(failed=0, attempted=6)